# Project Outline
1. Train license plate detection model
2. Implement blurring or replacement of license plate area 
3. Deploy on a server with fast.ai or fast api
4. Implement CI/CD pipeline for deployment
5. Create website that allows a user to upload images to be censored